In [1]:
# IMPORTS

import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# LOADING DATASET
data_path = "../data/MNIST/"
train = pd.DataFrame.to_numpy(pd.read_csv(f"{data_path}sign_mnist_train.csv"))
test = pd.DataFrame.to_numpy(pd.read_csv(f"{data_path}sign_mnist_test.csv"))

In [3]:
# TRAIN/TEST SPLIT
X_train = train[:,1:].reshape((train.shape[0], 28, 28, 1))
y_train = train[:,0]
X_test = test[:,1:].reshape((test.shape[0], 28, 28, 1))
y_test = test[:,0]

In [4]:
y_train

array([ 3,  6,  2, ..., 18, 17, 23], dtype=int64)

In [5]:
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.python.keras.layers import InputSpec, Layer

class Argmax(Layer):
    """
    Based on https://github.com/YerevaNN/R-NET-in-Keras/blob/master/layers/Argmax.py
    """
    def __init__(self, axis=-1, **kwargs):
        super(Argmax, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis

    def call(self, inputs, mask=None):
        return K.argmax(inputs, axis=self.axis)

    def compute_output_shape(self, input_shape):
        input_shape = list(input_shape)
        del input_shape[self.axis]
        return tuple(input_shape)

    def compute_mask(self, x, mask):
        return None

    def get_config(self):
        config = {'axis': self.axis}
        base_config = super(Argmax, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [6]:
# DEFINITION OF THE NEURAL NETWORK
inputs = tf.keras.Input(shape=(28,28,1,))
x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding="same", strides=1)(inputs)
x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", strides=1)(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1)(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(25, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(28,28,1,)),
	tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.MaxPool2D(),

	tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.MaxPool2D(),

	tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", strides=1),
	tf.keras.layers.MaxPool2D(),

	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(25, activation=tf.nn.softmax),
	#tf.keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.int64))
 	#tf.keras.layers.Lambda(lambda x: tf.argmax(x)),
	])


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.categorical_crossentropy,
               metrics=['accuracy'])


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 8)         80        
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 8)         584       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 8)        0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 16)        1168      
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                    

In [10]:
y_train_OneHot = np.zeros((y_train.shape[0], 25), dtype=int)
for i in range(y_train.shape[0]):
    y_train_OneHot[i][y_train[i]] = 1
y_test_OneHot = np.zeros((y_test.shape[0], 25), dtype=int)
for i in range(y_test.shape[0]):
    y_test_OneHot[i][y_test[i]] = 1

In [11]:
h = model.fit(X_train,y_train_OneHot,batch_size=32,epochs=10,)

Epoch 1/10
858/858 [==============================] - 11s 12ms/step - loss: 0.9899 - accuracy: 0.8369
Epoch 2/10
858/858 [==============================] - 10s 12ms/step - loss: 0.0192 - accuracy: 0.9961
Epoch 3/10
858/858 [==============================] - 10s 12ms/step - loss: 0.0463 - accuracy: 0.9858
Epoch 4/10
858/858 [==============================] - 10s 12ms/step - loss: 2.8670e-04 - accuracy: 1.0000
Epoch 5/10
858/858 [==============================] - 10s 12ms/step - loss: 1.1059e-04 - accuracy: 1.0000
Epoch 6/10
858/858 [==============================] - 10s 12ms/step - loss: 6.9435e-05 - accuracy: 1.0000
Epoch 7/10
858/858 [==============================] - 11s 13ms/step - loss: 4.7529e-05 - accuracy: 1.0000
Epoch 8/10
858/858 [==============================] - 11s 13ms/step - loss: 3.2309e-05 - accuracy: 1.0000
Epoch 9/10
858/858 [==============================] - 11s 12ms/step - loss: 2.1644e-05 - accuracy: 1.0000
Epoch 10/10
858/858 [==============================] - 12s

In [12]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)
y_testt = np.argmax(y_test_OneHot,axis=1)
print(np.sum(y_testt == preds)/len(preds))

225/225 [==============================] - 1s 5ms/step
0.8530395984383714


In [13]:
model.save('real_time_detection/modelMNIST.h5')